# Applying Machine Learning and Search Methods for S&P500 Stock Portfolio Forecasting and Optimization

 Project developed by: **Eduardo Passos** [202205630](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202205630), **Pedro Fernandes** [202208347](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202208347) and **Rafael Pacheco** [202206258](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202206258)

### Index {#index} #############################################
1. [Project Introduction](#intro)
2. [Introduction to Stock Concepts](#intro2)
3. [Data Extraction and Collection](#data)

    3.1 [Visualization and Formatting](#vis)
4. [Exploratory Data Analysis](#eda)

5. 

? [Conclusion](#conclusion)
? [References](#ref)

# Project Introduction and Motivation {#intro}

This project encompasses the creation of a well suited investment strategy based on the S&P500 stock dataset.
The highlights of the group's development process are detailed throughout this report. All of the specific files utilized during the project's development can be found inside the submitted folder.

In order to predict stock behaviour, we employed:

 - `Deep Learning`: Long Short-Term Memory (LSTM)
 - ...

To optimize portfolio selection, we implemented:

 - `Search Methods`: Monte Carlo Tree Search (MCTS)
 - ...

The stock market is highly volatile and unpredictable, making stock price prediction nearly luck based.

In order to create strategies that allow for investors to efficiently obtain risk-adjusted returns, we can use **S&P500 data** to get a better understanding of how the stock market may behave, based on previously collected data and statistics.

It's important to mention that it doesn't always follow a guaranteed predictable, mathematical pattern. It is influenced by many real-world factors, independent to a company's growth and significance.

# Introduction to Stock Concepts

In case the reader is unfamiliar with stocks and investing, we decided to briefly explain key concepts used throughout this report.

The S&P500 is a stock market index that tracks the performance of 500 of the largest publicly traded companies in the United States. As per requested in the project statement, we used this dataset's information, from 2010 to 2023, in order to predict the stock behaviour of those companies during January 20204.

### **What are stocks, and why are they an investment?**
Stocks (or shares) represent ownership in a company. Investors buy stocks to gain a portion of a company's profits, or to benefit from an increase in the stock's market value.

#### **What are tickers?**
A ticker is a unique symbol assigned to a company's stock, essentially an identifier for each company, in order to facilitate stock tracking:

 - `AAPL`: Apple Inc.
 - `GOOG`: Alphabet Inc. (or, simply put, Google)
 
### **What are opening and closing prices?**
The opening price is the price at which a stock begins trading, when the market opens for the day. 
The closing price is the actual last transaction price on that day, for that specific stock.

We will be using daily windows in order to predict these prices.

### **What are windows, and how are they helpful during prediction?**
In time series analysis, a "window" refers to a segment of the data used for analysis or prediction. 
By using time series analysis, we aim to identify patterns, trends, and seasonal effects in the data.

### **Most importantly, how can I gain or lose money by investing?**

A positive return indicates profit, while a negative return signifies a loss. 
These are typically expressed as a percentage of the original investment. 

Imagine the investor purchases stock at 100:

 - Stock price increases from 100 to 110 -> the return is 10% -> <span style="color:green">Profit!</span>

 - Stock price decreases from 100 to 90 -> the return is -10% -> <span style="color:red">Loss!</span>

Market fluctuations dictate stock prices, which in return represent profit or loss for investors.

# Data Extraction and Collection

In order to extract the 2010-2023 section of the dataset, we used the `yfinance` module. 

[Wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies) is also accessed to download a table containing a list of S&P500 tickers.

The functions below document the extraction and collection process.

In [14]:
import pandas as pd
import yfinance as yf
import os

In [15]:
# Step 1: Get the list of S&P 500 companies
def get_sp500_tickers():
    # Download the table from Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    tickers = table['Symbol'].tolist()
    
    # Remove any invalid ticker symbols if necessary
    tickers = [ticker.replace('.', '-') for ticker in tickers]  # For Yahoo Finance compatibility
    return tickers

In [16]:
# Step 2: Download data for each stock
def download_sp500_data(tickers, start_date="2010-01-01", end_date="2024-12-31", group_by="ticker"):
    data = {}
    for ticker in tickers:
        print(f"Downloading data for {ticker}...")
        try:
            data[ticker] = yf.download(ticker, start=start_date, end=end_date)
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
    return data

In [17]:
# Step 3: Save or analyze the data

raw_directory = "raw_csvs"  # Specify the directory where you want to save the files

def save_data_to_csv(data):
    # Create the directory if it doesn't exist
    if not os.path.exists(raw_directory):
        os.makedirs(raw_directory)
    
    for ticker, df in data.items():
        if not df.empty:
            # Save the file in the specified directory
            file_path = os.path.join(raw_directory, f"{ticker}.csv")
            df.to_csv(file_path)
            print(f"Data for {ticker} saved to {file_path}.")
        else:
            print(f"No data for {ticker}.")


Uncomment to run. (dps muda se tava farto de tirar e meter acentos)

```py 

tickers = get_sp500_tickers()
sp500_data = download_sp500_data(tickers)
save_data_to_csv(sp500_data)

```

## Visualization and Formatting


Let's visualize the data we extracted. We'll use the Tesla stocks, `TSLA`, for showcasing formats and changes, since each csv was extracted equally.

In [19]:
df = pd.read_csv('./raw_csvs/TSLA.csv')
df.head()

,Price,Adj Close,Close,High,Low,Open,Volume
0,Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
1,Date,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-06-29 00:00:00+00:00,1.5926669836044312,1.5926669836044312,1.6666669845581055,1.1693329811096191,1.2666670083999634,281494500
3,2010-06-30 00:00:00+00:00,1.5886670351028442,1.5886670351028442,2.0280001163482666,1.553333044052124,1.7193330526351929,257806500
4,2010-07-01 00:00:00+00:00,1.4639999866485596,1.4639999866485596,1.7280000448226929,1.3513330221176147,1.6666669845581055,123282000


In [20]:
df.columns

Index(['Price', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

The dataset is presented in an unusual format. In order to facilitate analysis and training, we will be converting it into something more familiar, taking into consideration that:
 - Ticker is consistent for each csv;
 - `Price` can be overwritten as `Date`;
 - Removing NaN from column 1 will finish the process;

The function below applies this process to any given dataset from the `raw_csvs` folder, transforming it into a clean, working dataframe.

This is done by opening each `csv` file and converting it to a dataframe using `pandas`. Then, the irrelevant rows (0 contains tickers, and 1 contains NaN) are dropped. The `Price` column is in place of `Date`, therefore, `Price` is changed to `Date`. Finally, the date + time format is modified to `YYYY-MM-DD` format, making sure we only keep relevant information. We also transform each string column into `float64`, except for `Volume`, which is always an Integer.


Optionally, we can keep only the `Adj Close` column for each stock, which is the adjusted closing price, and ultimatelly our target for prediction. If the argument **onlyAdj** is kept as `False`, then all of the clean columns are kept.

In [ ]:
def process_csv(file, onlyAdj=False):
    df = pd.read_csv(file)
    
    # Drop rows with index 0 and 1, weird download format
    df = df.drop([0, 1])
    df.reset_index(drop=True, inplace=True)
    
    df.rename(columns={'Price': 'Date', 'Adj Close': 'AdjClose'}, inplace=True) 
    df.rename(columns={'Adj Close': 'AdjClose'}, inplace=True) 
    
    # Get only date
    df['Date'] = pd.to_datetime(df['Date']).dt.date 
    
    if onlyAdj:
        # Keeps only the 'Date' and Adjusted Close ('Adj Close') columns
        df = df[['Date', 'Adj Close	']] 
 
    # Set 'Date' as the index 
    df.set_index('Date', inplace=True)
    
    # Reset the index so that 'Date' is no longer the index
    df.reset_index(inplace=True)

    # Finally, convert strings to float
    for col in ["AdjClose", "Close", "Open", "High", "Low"]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Check the current columns to see what needs to be dropped (debug)
    #print(df.columns)

    return df

In [22]:
df_clean = process_csv('./raw_csvs/TSLA.csv')
df_clean.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2010-06-29,1.5926669836044312,1.5926669836044312,1.6666669845581055,1.1693329811096191,1.2666670083999634,281494500
1,2010-06-30,1.5886670351028442,1.5886670351028442,2.0280001163482666,1.553333044052124,1.7193330526351929,257806500
2,2010-07-01,1.4639999866485596,1.4639999866485596,1.7280000448226929,1.3513330221176147,1.6666669845581055,123282000
3,2010-07-02,1.2799999713897705,1.2799999713897705,1.5399999618530273,1.24733304977417,1.5333329439163208,77097000
4,2010-07-06,1.0740000009536743,1.0740000009536743,1.3333330154418945,1.0553330183029175,1.3333330154418945,103003500


In order to **automate this process**, we can get a list of tickers and save clear data to the directory below.

In [23]:
tickers = get_sp500_tickers()
tickers.sort()

In [ ]:
clean_directory = 'clean_csvs'

def save_cleaned_data(tickers, raw_dir, clean_dir):
    if not os.path.exists(clean_dir):
        os.makedirs(clean_dir)
    
    for ticker in tickers:
        raw_file_path = os.path.join(raw_dir, f"{ticker}.csv")
        #print(raw_file_path)
        if os.path.exists(raw_file_path):
            # Process the CSV file
            #print(f"Processing {ticker}...")
            df = process_csv(raw_file_path)
            
            if not df.empty:
                # Save the processed DataFrame to the clean directory
                clean_file_path = os.path.join(clean_dir, f"{ticker}.csv")
                df.to_csv(clean_file_path, index=False)
                #print(f"Data for {ticker} saved to {clean_file_path}.")
            else:
                #print(f"No valid data for {ticker}. Skipping...")
                pass
        else:
            #print(f"File for {ticker} not found in {raw_dir}. Skipping...")
            pass

By running this line of code, every `csv` file in the `raw_csv` directory will be read and its new, formatted contents will be written to a new `csv` file in the `clean_csv` directory.

```py
save_cleaned_data(tickers, raw_directory, clean_directory)
```

# Exploratory Data Analysis 

Now that our data is presented in a familiar format, we can begin to analyze the `csv` contents.

The first step is to check if all files' data types are consistent with one anoteher, and verify if any of them possess any NaN values.

In [ ]:

has_nan = [] 

# Iterate over each file 
for ticker in tickers:
    file_path = os.path.join(clean_directory, f"{ticker}.csv")

    # Temporarily transform into dataframe
    df = pd.read_csv(file_path)

    #

IndentationError: unindent does not match any outer indentation level (<string>, line 4)